1. Loading the Dataset <br>
2. Data Preprocessing <br>
3. Rule 1 - Extracting sentences with President's mention. <br>
4. Rule 2 - Extracting the mentions of initiatives. <br>
5. Rule 3 - Relation Extraction Part 1 <br>
6. Rule 4 - Relation Extraction Part 2 <br>
7. Combining Rule 3 & Rule 4 <br>
8. Rule 5 - Patterns using Prepositions <br>

In [9]:
file_names=os.listdir(path='Converted sessions/')

In [10]:
len(file_names)

49

In [11]:
file_names[:5]

['Session 25 - 1970',
 'Session 26 - 1971',
 'Session 27 - 1972',
 'Session 28 - 1973',
 'Session 29 - 1974']

In [158]:
import glob
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [13]:
folders = glob.glob('Converted sessions/Session*')
folders

['Converted sessions\\Session 25 - 1970',
 'Converted sessions\\Session 26 - 1971',
 'Converted sessions\\Session 27 - 1972',
 'Converted sessions\\Session 28 - 1973',
 'Converted sessions\\Session 29 - 1974',
 'Converted sessions\\Session 30 - 1975',
 'Converted sessions\\Session 31 - 1976',
 'Converted sessions\\Session 32 - 1977',
 'Converted sessions\\Session 33 - 1978',
 'Converted sessions\\Session 34 - 1979',
 'Converted sessions\\Session 35 - 1980',
 'Converted sessions\\Session 36 - 1981',
 'Converted sessions\\Session 37 - 1982',
 'Converted sessions\\Session 38 - 1983',
 'Converted sessions\\Session 39 - 1984',
 'Converted sessions\\Session 40 - 1985',
 'Converted sessions\\Session 41 - 1986',
 'Converted sessions\\Session 42 - 1987',
 'Converted sessions\\Session 43 - 1988',
 'Converted sessions\\Session 44 - 1989',
 'Converted sessions\\Session 45 - 1990',
 'Converted sessions\\Session 46 - 1991',
 'Converted sessions\\Session 47 - 1992',
 'Converted sessions\\Session 48 -

In [27]:
# to check for regex patterns
testfiles = glob.glob('Converted sessions/Session 73 - 2018/USA*.txt')
testfiles

['Converted sessions/Session 73 - 2018\\USA_73_2018.txt']

In [30]:
testfiles[0].split('_')[-1].split('.')[-2]

'2018'

In [33]:
testfiles[0].split('_')[-2]

'73'

In [39]:
testfiles[0].split('_')[0].split('\\')[-1]

'USA'

In [44]:
df = pd.DataFrame(columns=['Country','Session','Year','Speech'])

In [45]:
# read speeches by USA
i=0
# for each folder
for file in folders:
    # getting the txt file containing the speech of USA
    speech = glob.glob(file+'/USA*.txt')
    # reading the txt file
    with open(speech[0],encoding='utf8') as f:
        # reading the speech
        df.loc[i,'Speech'] = f.read()
        # extracting the year of speech
        df.loc[i,'Year'] = speech[0].split('_')[-1].split('.')[0]
        # extracting the session
        df.loc[i,'Session'] = speech[0].split('_')[-2]
        # extracting the country name
        df.loc[i,'Country'] = speech[0].split('_')[0].split('\\')[-1]
        i += 1

In [46]:
df.head()

,Country,Session,Year,Speech
0,USA,25,1970,1.\t It is my privilege to extend to you once ...
1,USA,26,1971,"Mr. President, I should like first of all to ..."
2,USA,27,1972,During the past few years the world has made r...
3,USA,28,1973,﻿35.\tI should like to add my congratulations ...
4,USA,29,1974,"Last year, at the previous session [.2124th me..."


In [48]:
df['Speech'][1]

' Mr. President, I should like first of all to associate myself with the previous speakers who have congratulated you on your election to the high office of President of the twenty-sixth session of the General Assembly. The United States is confident that you will pursue the ideals of the Charter with the same dedication as your distinguished predecessors.\n\n2.\tThe United Nations is an important instrument of peace. It is also the repository of many of the world\'s hopes. We must ensure that it is true to those hopes; that we here assembled-the representatives of 130 nations- meet our high responsibilities not only to our own world but to the world of the future.\n3.\tBuilding a structure of peace-a structure that will stand-requires patience, dedication and realism. Working at peace working at it seriously means more than long hours of careful deliberation. It means putting aside preconceptions; it means weighing carefully what our real interests are, not only as citizens of our own

In [49]:
# function to preprocess speech
def clean(text):
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing newline characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub('-',' ',str(text))
    text = re.sub('- ','',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub('Mr\.','Mr',str(text))
    text = re.sub('Mrs\.','Mrs',str(text))
    # removing references to outside text enclosed in [] brackets
    text = re.sub("[\(\[].*?[\)\]]",'',str(text))
    return text

In [50]:
# preprocessing speeches
df['Speech_clean'] = df['Speech'].apply(clean)

In [51]:
df.head()

,Country,Session,Year,Speech,Speech_clean
0,USA,25,1970,1.\t It is my privilege to extend to you once ...,It is my privilege to extend to you once agai...
1,USA,26,1971,"Mr. President, I should like first of all to ...","Mr President, I should like first of all to a..."
2,USA,27,1972,During the past few years the world has made r...,During the past few years the world has made r...
3,USA,28,1973,﻿35.\tI should like to add my congratulations ...,﻿I should like to add my congratulations to th...
4,USA,29,1974,"Last year, at the previous session [.2124th me...","Last year, at the previous session , in my fir..."


In [52]:
df['Speech_clean'][1]

" Mr President, I should like first of all to associate myself with the previous speakers who have congratulated you on your election to the high office of President of the twenty sixth session of the General Assembly. The United States is confident that you will pursue the ideals of the Charter with the same dedication as your distinguished predecessors.  The United Nations is an important instrument of peace. It is also the repository of many of the world's hopes. We must ensure that it is true to those hopes; that we here assembled the representatives of 130 nations  meet our high responsibilities not only to our own world but to the world of the future. Building a structure of peace a structure that will stand requires patience, dedication and realism. Working at peace working at it seriously means more than long hours of careful deliberation. It means putting aside preconceptions; it means weighing carefully what our real interests are, not only as citizens of our own countries, b

In [53]:
# splitting the sentences
def sentences(text):
    # split sentences & questions
    text = re.split('[.?]',text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

In [54]:
df['Sentence'] = df['Speech_clean'].apply(sentences)

In [55]:
df.head()

,Country,Session,Year,Speech,Speech_clean,Sentence
0,USA,25,1970,1.\t It is my privilege to extend to you once ...,It is my privilege to extend to you once agai...,[ It is my privilege to extend to you once aga...
1,USA,26,1971,"Mr. President, I should like first of all to ...","Mr President, I should like first of all to a...","[ Mr President, I should like first of all to ..."
2,USA,27,1972,During the past few years the world has made r...,During the past few years the world has made r...,[During the past few years the world has made ...
3,USA,28,1973,﻿35.\tI should like to add my congratulations ...,﻿I should like to add my congratulations to th...,[﻿I should like to add my congratulations to t...
4,USA,29,1974,"Last year, at the previous session [.2124th me...","Last year, at the previous session , in my fir...","[Last year, at the previous session , in my fi..."


In [56]:
df['Sentence'][1]

[' Mr President, I should like first of all to associate myself with the previous speakers who have congratulated you on your election to the high office of President of the twenty sixth session of the General Assembly',
 ' The United States is confident that you will pursue the ideals of the Charter with the same dedication as your distinguished predecessors',
 '  The United Nations is an important instrument of peace',
 " It is also the repository of many of the world's hopes",
 ' We must ensure that it is true to those hopes; that we here assembled the representatives of 130 nations  meet our high responsibilities not only to our own world but to the world of the future',
 ' Building a structure of peace a structure that will stand requires patience, dedication and realism',
 ' Working at peace working at it seriously means more than long hours of careful deliberation',
 ' It means putting aside preconceptions; it means weighing carefully what our real interests are, not only as cit

In [57]:
# create a dataframe containing sentences
df2 = pd.DataFrame(columns=['Sentence','Year','Len'])
row_list=[]
# for each speech
for i in range(len(df)):
    # for each sentence
    for sent in df.loc[i,'Sentence']:
        # calculate word count in a sentence
        wordcount = len(sent.split())
        # extract year of speech
        year = df.loc[i,'Year']
        dict1 = {'Year':year,'Sentence':sent,'Len':wordcount}
        row_list.append(dict1)
df2 = pd.DataFrame(row_list)

In [58]:
df2.shape

(8948, 3)

In [59]:
df2.head()

,Year,Sentence,Len
0,1970,It is my privilege to extend to you once agai...,33
1,1970,Your country has few peers in the contributio...,17
2,1970,Your father was a leader in the League of Nat...,29
3,1970,In the United Nations community we have long ...,65
4,1970,We are indeed fortunate to have you to lead u...,23


## Rule 1

In [63]:
from spacy.matcher import Matcher
import spacy
# ner is entity recognizer (doc.ents) & textcat is text categorizer
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [106]:
sample_sent1 = "President Moon of South Korea. President Moon to announce."
sample_sent2 = "Lost a second President of China."
doc_samp = nlp(sample_sent2)
for token in doc_samp:
    print(token.text,'=>',token.pos_)

Lost => VERB
a => DET
second => ADJ
President => NOUN
of => ADP
China => PROPN
. => PUNCT


In [71]:
# function to find sentences mentioning President's name
def find_names(text):
    names=[]
    doc=nlp(text)
    pattern=[{'LOWER':'president'},{'POS':'ADP','OP':'?'},{'POS':'PROPN'}]
    matcher=Matcher(nlp.vocab)
    matcher.add('Names',[pattern])
    matches=matcher(doc)
    # finding patterns in the text
    for i in range(len(matches)):
        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        names.append(str(token))
    return names

In [72]:
# extracting sentences with President's mentions
extracted_sentences = []
for i in range(df2.shape[0]):
    extracted_sentences.append(find_names(df2['Sentence'][i]))
# storing the extracted sentences in a dataframe
df2['President_Names'] = extracted_sentences

In [ ]:
sentences_president = []
for i in range(df2.shape[0]):
    if df2['President_Names'][i] != []:
        sentences_president.append(df2['Sentence'][i])
sentences_president

In [76]:
# number of sentences containing President's mention
len(sentences_president)

106

In [77]:
sentences_president[1]

' While recent developments in the area, including the tragic death of President Nasser, have cast a shadow over the high hopes of just a few weeks ago that a beginning toward that goal had been made, those hopes have not been extinguished'

In [73]:
df2.head()

,Year,Sentence,Len,President_Names
0,1970,It is my privilege to extend to you once agai...,33,[]
1,1970,Your country has few peers in the contributio...,17,[]
2,1970,Your father was a leader in the League of Nat...,29,[]
3,1970,In the United Nations community we have long ...,65,[]
4,1970,We are indeed fortunate to have you to lead u...,23,[]


In [74]:
# extracting only the non-null outputs
mention = []
for i in range(df2.shape[0]):
    if df2['President_Names'][i] != []:
        mention.append(df2['President_Names'][i])
mention

[['President Nixon'],
 ['President Nasser'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President Nixon'],
 ['President of Mexico'],
 ['President Ford'],
 ['President Ford'],
 ['President Ford'],
 ['President Ford'],
 ['President Ford'],
 ['President Ford'],
 ['President of Lebanon'],
 ['President Ford'],
 ['President Carter'],
 ['President Carter'],
 ['President Carter'],
 ['President Carter'],
 ['President Mondaleat'],
 ['President Carter', 'President in January'],
 ['President Carter'],
 ['President Carter'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Reagan'],
 ['President Dwight'],
 ['President of Brazil'],
 ['President Sarney'],
 

## Rule 2

Sample Sentences from dataset
1. The US will continue its strong efforts to advance **the United Nations plan** for Namibia.
2. The US & Italy have proposed **a Global Peace Operations Initiative**.
3. For 35 years, **the North Atlantic alliance** has guaranteed the peace in Europe.

In [117]:
sample_sent3 = "a Global Peace Operations Initiative"
doc_samp = nlp(sample_sent3)
for token in doc_samp:
    print(token.text,'=>',token.pos_,'=>',token.dep_)

a => DET => det
Global => PROPN => compound
Peace => PROPN => compound
Operations => PROPN => compound
Initiative => PROPN => ROOT


In [122]:
doc_samp[0].pos

90

In [110]:
def prog_sent_test(text):
    patterns = [r'(?i)\bplan\b',r'(?i)\bprogramme\b']
    flag=0
    for pat in patterns:
        if re.search(pat,text) != None:
            flag=1
            break
    return flag
res = prog_sent_test("Global Peace Operations ProgrAMME")
print(res)

1


In [92]:
# to check if keywords like 'programs', 'schemes', etc are present in sentences
# ?i is a regex flag for case-insensitive; it's a global flag, so it should be at the start of regex pattern.
# \b is for non-word boundary. Anything except letters, digits & underscore is a non-word.
def prog_sent(text):
    patterns = [r'(?i)\bplan\b',r'(?i)\bprogramme\b',r'(?i)\bscheme\b',r'(?i)\bcampaign\b',r'(?i)\binitiative\b',
                r'(?i)\bconference\b',r'(?i)\bagreement\b',r'(?i)\balliance\b']
    flag=0
    for pat in patterns:
        if re.search(pat,text) != None:
            flag=1
            break
    return flag

In [93]:
df2['Check_Schemes'] = df2['Sentence'].apply(prog_sent)

In [94]:
df2.head()

,Year,Sentence,Len,President_Names,Check_Schemes
0,1970,It is my privilege to extend to you once agai...,33,[],0
1,1970,Your country has few peers in the contributio...,17,[],0
2,1970,Your father was a leader in the League of Nat...,29,[],0
3,1970,In the United Nations community we have long ...,65,[],0
4,1970,We are indeed fortunate to have you to lead u...,23,[],0


In [133]:
# extract initiatives using pattern matching
def all_schemes(text,check):
    schemes = []
    doc = nlp(text)
    prog_list = ['programme','scheme','initiative','campaign','agreement','conference','alliance','plan']
    # pattern to match initiative names
    pattern = [{'POS':'DET'},
               {'POS':'PROPN','DEP':'compound'},
               {'POS':'PROPN','DEP':'compound'},
               {'POS':'PROPN','OP':'?'},
               {'POS':'PROPN','OP':'?'},
               {'LOWER':{'IN':prog_list},'OP':'+'}]
    if check==0:
        # return blank list
        return schemes
    # Matcher class object
    matcher = Matcher(nlp.vocab)
    matcher.add('matching',[pattern])
    matches = matcher(doc)
    for i in range(len(matches)):
        # match: id, start, end
        start,end=matches[i][1]+1,matches[i][2]
        #if doc[start].pos_ == 'DET':
        #    start=start+1
        # matched string
        span = str(doc[start:end])
        if (len(schemes)!=0) and (schemes[-1] in span):
            schemes[-1] = ''
        else:
            schemes.append(span)
    return schemes

In [134]:
df2['Schemes1'] = df2.apply(lambda x:all_schemes(x.Sentence,x.Check_Schemes),axis=1)

In [147]:
# extracting only the non-null outputs
mention = []
for i in range(df2.shape[0]):
    if df2['Schemes1'][i] != []:
        mention.append(df2['Schemes1'][i])
mention

[['United States initiative'],
 ['United Nations Conference'],
 ['Suez Canal agreement'],
 ['World Food Conference'],
 ['World Population Conference', 'World Food Conference'],
 ['Third United Nations Conference'],
 ['Geneva Peace Conference'],
 ['Geneva Peace Conference'],
 ['Bonn Economic Summit Conference'],
 ['Bonn Economic Summit Conference'],
 ['World Food Conference'],
 ['World Food Conference'],
 ['United Nations Conference'],
 ['Second Review Conference'],
 ['United Nations Conference'],
 ['United Nations plan'],
 ['United Nations plan'],
 ['Caribbean Basin Initiative'],
 ['North Atlantic alliance'],
 ['San Francisco Conference'],
 ['Third Review Conference'],
 ['San Francisco Conference'],
 ['North American Free Trade Agreement'],
 ['North American Free Trade Agreement'],
 ['United Nations Conference'],
 ['Good Friday Agreement'],
 ['United Nations Development Programme', 'World Food Programme'],
 ['African Crisis Response Initiative'],
 ['World Food Programme'],
 ['Prolifera

In [98]:
# extracting sentences which mention some initiatives
initiatives = []
for i in range(df2.shape[0]):
    if df2['Schemes1'][i] != []:
        initiatives.append(df2['Sentence'][i])
initiatives

[' Security Council resolution 242  stands as a beacon pointing the way to peace, and the parties concerned, in accepting the United States initiative of June, have committed themselves to carry out that resolution in all its parts',
 ' Already the plans for the United Nations Conference on the Human Environment in 1972 have begun to take shape and have helped to focus the attention of United Nations Member Governments on this worldwide challenge; but we should not wait for the Stockholm Conference before launching necessary initiatives',
 ' A fair approach should be founded on two basic principles:  That a Suez Canal agreement is merely a step towards complete and full implementation of resolution 242  within a reasonable period of time, and not an end in itself that has to be clearly established in any agreement; also  That neither side can realistically expect \\to achieve, as part of an interim agreement, complete agreement on the terms and conditions of an over all settlement  if 

In [129]:
len(initiatives)

36

In [100]:
initiatives[0]

' Security Council resolution 242  stands as a beacon pointing the way to peace, and the parties concerned, in accepting the United States initiative of June, have committed themselves to carry out that resolution in all its parts'

## Rule 3: Noun (subject), Verb, Noun (object)

Sample sentences from database:
1. The United **States** will **support** these **principles**.
2. Yet these very small **entities need** more than most the **assistance** that the United Nations system can provide.
3. I have proposed to Congress that the United **States provide** additional **funding** for our work in Iraq, the greatest financial commitment of its kind since the Marshall Plan. 

In [132]:
from spacy import displacy
displacy.render(nlp("I have proposed to Congress that the United States provide additional funding for our work in Iraq, the greatest financial commitment of its kind since the Marshall Plan. "),jupyter=True)

In [148]:
def rule3(text):
    doc = nlp(text)
    sent = []
    for token in doc:
        if (token.pos_=='VERB'):
            phrase = ''
            for sub_tok in token.lefts:
                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    phrase += sub_tok.text
                    phrase += ' '+token.lemma_+' '
                    for sub_tok in token.rights:
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                            phrase += sub_tok.text
                            sent.append(phrase)
    return sent

In [152]:
# create a df containing sentence and its output for rule 3
row_list = []
for i in range(len(df2)):
    sent = df2.loc[i,'Sentence']
    year = df2.loc[i,'Year']
    output = rule3(sent)
    dict1 = {'Year':year,'Sentence':sent,'Output':output}
    row_list.append(dict1)

df_rule3 = pd.DataFrame(row_list)

In [154]:
df_rule3.head()

,Year,Sentence,Output
0,1970,It is my privilege to extend to you once agai...,[]
1,1970,Your country has few peers in the contributio...,[country have peers]
2,1970,Your father was a leader in the League of Nat...,[you add chapter]
3,1970,In the United Nations community we have long ...,[]
4,1970,We are indeed fortunate to have you to lead u...,[]


In [177]:
# for non-empty output rows
df_rule3_show = pd.DataFrame(columns=df_rule3.columns)
for row in range(len(df_rule3)):
    if len(df_rule3.loc[row,'Output'])!=0:
        df_rule3_show = df_rule3_show.append(df_rule3.loc[row,:])
df_rule3_show.reset_index(inplace=True)
df_rule3_show.drop('index',axis=1,inplace=True) 
df_rule3_show.head()

,Year,Sentence,Output
0,1970,Your country has few peers in the contributio...,[country have peers]
1,1970,Your father was a leader in the League of Nat...,[you add chapter]
2,1970,It will care a great deal what we do about th...,[It care deal]
3,1970,Unless and until the Members of the United Na...,[Members demonstrate willingness]
4,1970,"In the view of the United States, the most cr...",[which face founders]


In [178]:
df_rule3_show.shape

(4021, 3)

In [164]:
# separate subject, verb & object
verb_dict = dict()
dis_dict = dict()
dis_list = []

for i in range(len(df_rule3_show)):
    sentence = df_rule3_show.loc[i,'Sentence']
    year = df_rule3_show.loc[i,'Year']
    output = df_rule3_show.loc[i,'Output']
    for sent in output:
        
        # separate subject, verb and object
        n1, v, n2 = sent.split()[0], sent.split()[1], sent.split()[2]
        
        # append to list, along with the sentence
        dis_dict = {'Sent':sentence,'Year':year,'Noun1':n1,'Verb':v,'Noun2':n2}
        dis_list.append(dis_dict)
        
        # counting the number of sentences containing the verb
        verb = sent.split()[1]
        if verb in verb_dict:
            verb_dict[verb]+=1
        else:
            verb_dict[verb]=1

df_sep = pd.DataFrame(dis_list)

In [165]:
df_sep.head()

,Sent,Year,Noun1,Verb,Noun2
0,Your country has few peers in the contributio...,1970,country,have,peers
1,Your father was a leader in the League of Nat...,1970,you,add,chapter
2,It will care a great deal what we do about th...,1970,It,care,deal
3,Unless and until the Members of the United Na...,1970,Members,demonstrate,willingness
4,"In the view of the United States, the most cr...",1970,which,face,founders


In [166]:
# top 10 most occuring verbs
df_sep['Verb'].value_counts()[:10]

have       350
take       162
make       140
support     98
give        92
see         80
seek        59
provide     57
face        57
build       53
Name: Verb, dtype: int64

In [186]:
# selecting a verb and looking at its pattern
df_support = df_sep[df_sep['Verb']=='support']
for index, sent in enumerate(df_support["Sent"][:7]):
    print(index+1,".",sent)

1 .  The United States strongly supports the strategy which is before this Assembly  and intends to participate fully in this common enterprise
2 .  We support his efforts
3 .  And, in recognition of the importance of trade to development, we are actively supporting the participation of Latin American and other developing countries in the coming negotiations on a new international monetary system
4 .  From the start the United States has supported that effort
5 .  In this spirit also, we support the permanent membership of Japan in the Security Council
6 .  We support the United Nations efforts in the fields of international law and human rights
7 .  The United States supports the dual entry of both South and North Korea into the United Nations without prejudice to their eventual reunification


## Rule 4: Adjective Noun structure

In [173]:
def rule4(text):
    doc = nlp(text)
    pat = []
    for token in doc:
        phrase = ''
        if (token.pos_ == 'NOUN') and (token.dep_ in ['dobj','pobj','nsubj','nsubjpass']):
            for subtoken in token.children:
                if (subtoken.pos_ == 'ADJ') or (subtoken.dep_ == 'compound'):
                    phrase += subtoken.text + ' '                  
            if len(phrase)!=0:
                phrase += token.text             
        if  len(phrase)!=0:
            pat.append(phrase)        
    return pat

In [175]:
# create a df containing sentence and its output for rule 4
row_list = []
for i in range(len(df2)):
    sent = df2.loc[i,'Sentence']
    year = df2.loc[i,'Year']
    output = rule4(sent)
    dict1 = {'Year':year,'Sent':sent,'Output':output}
    row_list.append(dict1)

df_rule4 = pd.DataFrame(row_list)

In [176]:
df_rule4.head()

,Year,Sent,Output
0,1970,It is my privilege to extend to you once agai...,"[States delegation, fifth session]"
1,1970,Your country has few peers in the contributio...,[few peers]
2,1970,Your father was a leader in the League of Nat...,"[new chapter, family tradition]"
3,1970,In the United Nations community we have long ...,"[Nations community, able spokesman, very begin..."
4,1970,We are indeed fortunate to have you to lead u...,[]


In [180]:
# for non-empty output rows
df_rule4_show = pd.DataFrame(columns=df_rule4.columns)
for row in range(len(df_rule4)):
    if len(df_rule4.loc[row,'Output'])!=0:
        df_rule4_show = df_rule4_show.append(df_rule4.loc[row,:])
df_rule4_show.reset_index(inplace=True, drop=True)
df_rule4_show.head()

,Year,Sent,Output
0,1970,It is my privilege to extend to you once agai...,"[States delegation, fifth session]"
1,1970,Your country has few peers in the contributio...,[few peers]
2,1970,Your father was a leader in the League of Nat...,"[new chapter, family tradition]"
3,1970,In the United Nations community we have long ...,"[Nations community, able spokesman, very begin..."
4,1970,During this session we shall commemorate with...,"[fitting ceremony, creative events]"


In [181]:
df_rule4_show.shape

(6170, 3)

In [183]:
for index, sent in enumerate(df_rule4_show["Sent"][:7]):
    print(index+1,".",sent)

1 .  It is my privilege to extend to you once again the warm congratulations of the United States delegation on your election to the Presidency of the twenty fifth session of the General Assembly
2 .  Your country has few peers in the contributions it has made to the work of this Organization
3 .  Your father was a leader in the League of Nations, and now, in this session of the General Assembly, you are adding a new chapter in the family tradition
4 .  In the United Nations community we have long known you as an able spokesman of your country, as a participant in the life of the United Nations from its very beginning and an authority on the Charter, as a leading scholar and practitioner in international law, and as one whose personal outlook is strongly humanitarian and imbued with hope and determination for a better future
5 .  During this session we shall commemorate with fitting ceremony one of the most creative and hopeful events of this century, the founding of the United Nations

## Combining Rule 3 & Rule 4

Sample Sentences
1. With support from many countries, **we** have **made impressive progress**.
2. **It** will **require military planners** the world over to recognize that training for peace operations is a legitimate part of every national security strategy. 

In [187]:
# modify rule 4
def rule4_modify(text,index):
    doc=nlp(text)
    phrase = ''
    for token in doc:
        if token.i == index:
            for subtoken in token.children:
                if (subtoken.pos_ == 'ADJ'):
                    phrase += ' '+subtoken.text
            break  
    return phrase

In [188]:
# combining rule 3 & rule 4
def rule3_modify(text):
    doc = nlp(text)
    sent = []
    for token in doc:
        if (token.pos_=='VERB'):
            phrase =''
            for sub_tok in token.lefts:
                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    # look for subject modifier
                    adj = rule4_modify(text,sub_tok.i)
                    phrase += adj + ' ' + sub_tok.text
                    # save the root word of the word
                    phrase += ' '+token.lemma_ 
                    # check for noun or pronoun direct objects
                    for sub_tok in token.rights:
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                            # look for object modifier
                            adj = rule4_modify(text,sub_tok.i)
                            phrase += adj+' '+sub_tok.text
                            sent.append(phrase)
    return sent

In [189]:
# create a df containing sentence and its output for modified rule 
row_list = []
# df2 contains all the sentences from all the speeches
for i in range(len(df2)):
    sent = df2.loc[i,'Sentence']
    year = df2.loc[i,'Year']
    output = rule3_modify(sent)
    dict1 = {'Year':year,'Sent':sent,'Output':output}
    row_list.append(dict1)
    
df_rule3_4 = pd.DataFrame(row_list)

In [190]:
df_rule3_4.head()

,Year,Sent,Output
0,1970,It is my privilege to extend to you once agai...,[]
1,1970,Your country has few peers in the contributio...,[ country have few peers]
2,1970,Your father was a leader in the League of Nat...,[ you add new chapter]
3,1970,In the United Nations community we have long ...,[]
4,1970,We are indeed fortunate to have you to lead u...,[]


In [191]:
# for non-empty output rows
df_show_mod = pd.DataFrame(columns=df_rule3_4.columns)
for row in range(len(df_rule3_4)):
    if len(df_rule3_4.loc[row,'Output'])!=0:
        df_show_mod = df_show_mod.append(df_rule3_4.loc[row,:])
df_show_mod.reset_index(inplace=True, drop=True)
df_show_mod.head()

,Year,Sent,Output
0,1970,Your country has few peers in the contributio...,[ country have few peers]
1,1970,Your father was a leader in the League of Nat...,[ you add new chapter]
2,1970,It will care a great deal what we do about th...,[ It care great deal]
3,1970,Unless and until the Members of the United Na...,[ Members demonstrate willingness]
4,1970,"In the view of the United States, the most cr...",[ which face founders]


In [192]:
df_show_mod.shape

(4021, 3)

In [193]:
for index, sent in enumerate(df_show_mod["Sent"][:7]):
    print(index+1,".",sent)

1 .  Your country has few peers in the contributions it has made to the work of this Organization
2 .  Your father was a leader in the League of Nations, and now, in this session of the General Assembly, you are adding a new chapter in the family tradition
3 .  It will care a great deal what we do about the future
4 .  Unless and until the Members of the United Nations demonstrate their willingness to deal effectively with these problems, the confidence of the world's peoples in the United Nations will remain uncertain and precarious
5 .  In the view of the United States, the most critical international problem today is the same as that which faced the founders of the United Nations: the need to build a reliable framework for peace and security among nations
6 .  We have sought, in the spirit of the Charter, to pursue peaceful settlement of all conflicts and confrontations, and we have urged others to do likewise
7 .  If at last we can check the nuclear arms race, the world will have b

## Rule 5: Patterns using Prepositions

Sample Sentences
1. We all believe the **benefits of globalization** must be allocated more broadly within and among socities.
2. As the Millennium Summit reflected, we have no **shortage of** worthy **goals**.
3. It should preserve the special **responsibility for peacekeeping** of Security Council permanent members.

In [195]:
displacy.render(nlp('''As the Millennium Summit reflected, 
we have no shortage of worthy goals.'''),style='dep',jupyter=True)

In [196]:
def rule5(text):
    doc=nlp(text)
    sent=[]
    for token in doc:
        if token.pos_=='ADP':
            phrase=''
            if token.head.pos_=='NOUN':
                phrase += token.head.text
                phrase += ' '+token.text
                for right_tok in token.rights:
                    if (right_tok.pos_ in ['NOUN','PROPN']):
                        phrase += ' '+right_tok.text
                if len(phrase)>2:
                    sent.append(phrase)
    return sent

In [197]:
# create a df containing sentence and its output for rule 5
row_list = []
# df2 contains all the sentences from all the speeches
for i in range(len(df2)):
    sent = df2.loc[i,'Sentence']
    year = df2.loc[i,'Year']
    output = rule5(sent)
    dict1 = {'Year':year,'Sent':sent,'Output':output}
    row_list.append(dict1)
    
df_rule5 = pd.DataFrame(row_list)

In [198]:
# for non-empty output rows
df_show5 = pd.DataFrame(columns=df_rule5.columns)
for row in range(len(df_rule5)):
    if len(df_rule5.loc[row,'Output'])!=0:
        df_show5 = df_show5.append(df_rule5.loc[row,:])
df_show5.reset_index(inplace=True, drop=True)
df_show5.head()

,Year,Sent,Output
0,1970,It is my privilege to extend to you once agai...,"[congratulations of delegation, congratulation..."
1,1970,Your country has few peers in the contributio...,"[peers in contributions, work of Organization]"
2,1970,Your father was a leader in the League of Nat...,"[leader in League, session of Assembly, chapte..."
3,1970,In the United Nations community we have long ...,"[spokesman of country, participant in life, li..."
4,1970,We are indeed fortunate to have you to lead u...,[strengthening of Nations]


In [200]:
# separate noun, preposition and noun

prep_dict = dict()
dis_dict = dict()
dis_list = []

for i in range(len(df_show5)):
    sentence = df_show5.loc[i,'Sent']
    year = df_show5.loc[i,'Year']
    output = df_show5.loc[i,'Output']
    for sent in output:
        n1, p, n2 = sent.split()[0], sent.split()[1], sent.split()[2:]
        dis_dict = {'Sent':sentence,'Year':year,'Noun1':n1,'Preposition':p,'Noun2':n2}
        dis_list.append(dis_dict)
        prep = sent.split()[1]
        if prep in prep_dict:
            prep_dict[prep]+=1
        else:
            prep_dict[prep]=1

df_sep5= pd.DataFrame(dis_list)

In [201]:
df_sep5.head()

,Sent,Year,Noun1,Preposition,Noun2
0,It is my privilege to extend to you once agai...,1970,congratulations,of,[delegation]
1,It is my privilege to extend to you once agai...,1970,congratulations,on,[election]
2,It is my privilege to extend to you once agai...,1970,congratulations,to,[Presidency]
3,It is my privilege to extend to you once agai...,1970,session,of,[Assembly]
4,Your country has few peers in the contributio...,1970,peers,in,[contributions]


In [202]:
# top 10 prepositions
df_sep5['Preposition'].value_counts()[:10]

of         6110
in         1212
for         892
to          568
on          344
with        251
between     202
from        140
as          121
against     117
Name: Preposition, dtype: int64

In [205]:
df_sep5[df_sep5['Preposition']=='against'].head(10)

,Sent,Year,Noun1,Preposition,Noun2
211,My Government continues strictly to observe r...,1970,sanctions,against,[regime]
691,This fact has often been cited in attacks aga...,1972,attacks,against,[]
728,The United Nations can also make a substantia...,1972,fight,against,[traffic]
777,"But political passion, however deeply held, c...",1972,violence,against,[persons]
778,Certainly the terrorist acts I have cited are...,1972,attacks,against,[fabric]
798,The draft articles on the prevention and puni...,1972,crimes,against,[agents]
819,These actions would mark a major advance in ...,1972,struggle,against,[terrorism]
1109,"The world community, therefore, must work urg...",1974,safeguards,against,[diversion]
1117,The United States will urge IAEA to draft an ...,1974,security,against,[theft]
1423,Procedural decisions can be based on the reco...,1975,violation,against,[State]


In [204]:
# selecting a preposition and looking at its pattern
df_against = df_sep5[df_sep5['Preposition']=='against']
for index, sent in enumerate(df_against["Sent"][:7]):
    print(index+1,".",sent)

1 .  My Government continues strictly to observe resolutions in regard to sanctions against the illegal regime in Rhodesia and in regard to the sale of arms to South Africa
2 .  This fact has often been cited in attacks against it
3 .  The United Nations can also make a substantial contribution in the fight against drug traffic, particularly heroin traffic
4 .  But political passion, however deeply held, cannot be a justification for criminal violence against innocent persons
5 .  Certainly the terrorist acts I have cited are totally unacceptable attacks against the very fabric of international order
6 .  The draft articles on the prevention and punishment of crimes against diplomatic agents and other internationally protected persons are already before the Assembly in the report of the International Law Commission 
7 .   These actions would mark a major advance in the struggle against international terrorism
